# Using the Fashion MNSIT dataset, get a accuracy of at least .8

Build a Neural Network from scratch and tune the hyperparameters to get an accuracy of at least 80% or more

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
# Importing the Fashion MNSIT dataset
(trainImages, trainLabels), (testImages, testLabels) = tf.keras.datasets.fashion_mnist.load_data()

The Labels names and their indexes

<table>
  <tr>
    <th>Label</th>
    <th>Class</th>
  </tr>
  <tr>
    <td>0</td>
    <td>T-shirt/top</td>
  </tr>
  <tr>
    <td>1</td>
    <td>Trouser</td>
  </tr>
    <tr>
    <td>2</td>
    <td>Pullover</td>
  </tr>
    <tr>
    <td>3</td>
    <td>Dress</td>
  </tr>
    <tr>
    <td>4</td>
    <td>Coat</td>
  </tr>
    <tr>
    <td>5</td>
    <td>Sandal</td>
  </tr>
    <tr>
    <td>6</td>
    <td>Shirt</td>
  </tr>
    <tr>
    <td>7</td>
    <td>Sneaker</td>
  </tr>
    <tr>
    <td>8</td>
    <td>Bag</td>
  </tr>
    <tr>
    <td>9</td>
    <td>Ankle boot</td>
  </tr>
</table>

In [ ]:
# Defining the Class names

classNames = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# Exploring the Training Dataset
print("Training Dataset shape: ", trainImages.shape)

# Training Dataset Labels
print("Dataset Lables: ", trainLabels)
print("Labels Length: ", len(trainLabels))

# Exploring the Testing Dataset
print("Testing Dataset shape: ", testImages.shape)

In [ ]:
# Preprocessing the Data

_index = 1000 # 59999 is the Max Index

plt.figure()
plt.imshow(trainImages[_index])
plt.colorbar()
plt.grid(False)
plt.show()

In [ ]:
# Verifying the format of the data and plotting it

plt.figure(figsize=(10, 10))

for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(trainImages[i], cmap=plt.cm.binary)
    plt.xlabel(classNames[trainLabels[i]])
 
plt.show()

In [6]:
# Flattening the Training Images and Testing Images
trainImages = trainImages.reshape(trainImages.shape[0], -1) / 255.0
testImages = testImages.reshape(testImages.shape[0], -1) / 255.0

In [7]:
# one-hot encoding the training and testing dataset

def one_hit_encode(outputLayerLength, dataLabel):
    return np.eye(dataLabel)[outputLayerLength]

trainLabelsEncoded = one_hit_encode(trainLabels, len(classNames))
testLabelEncoded = one_hit_encode(testLabels, len(classNames))

In [8]:
# The Neural Network Definition

class NeuralNetwork:
    def __init__(self, inputSize, hiddenSize, outputSize, learnRate=0.01, lambdaL1=0.0, lambdaL2=0.0):
         # Weights and Biases of the connection between first [0] and second [1] layer
         self.W1 = np.random.randn(inputSize, hiddenSize) * 0.01
         self.b1 = np.zeros((1, hiddenSize))
         
         # Weights and Biases of the connection between second [1] and third [2] layer
         self.W2 = np.random.randn(hiddenSize, outputSize) * 0.01
         self.b2 = np.zeros((1, outputSize))
         
         # Learning Rate to decrease error
         self.learnRate = learnRate
         
         # Regularizations through lambdas
         self.lambdal1 = lambdaL1
         self.lambdal2 = lambdaL2
        
    # axis=1 -- Ensures the operation is being done in row-wise, i.e, only one one axis
    # keepdims=Ture -- Ensures the output array is the same size as the input array
        
    def SoftMax(self, trainData):
        ePowerZi = np.exp(trainData - np.max(trainData, axis=1, keepdims=True))       # Subtracting the max to ensure the maximum limit of the exponent is (e^0 = 1)
        return ePowerZi / np.sum(ePowerZi, axis=1, keepdims=True)
    
    def ForwardPropogation(self, trainData):
        # The Linear Combination of the Weights and Biases for Hidden Layer -- np.dot() - Matrix Multiplcation
        self.Z1 = np.dot(trainData, self.W1) + self.b1
        self.A1 = np.tanh(self.Z1) # Plugging in the Activation function
        
        # The Linear Combination of the Weights and Biases for Output Layer -- np.dot() - Matrix Multiplcation
        self.Z2 = np.dot(self.A1, self.W2) + self.b2
        self.A2 = self.SoftMax(self.Z2)
        
        return self.A2
        
    def BackwardPropogation(self, trainData, labelData, predictionData):
        trainDataSize = trainData.shape[0]  # The Training Dataset size
        
        outputError = predictionData - labelData
        hiddenError = np.dot(outputError, self.W2.T) * (1 - np.tanh(self.Z1) ** 2)
        
        dW2 = np.dot(self.A1.T, outputError) / trainDataSize    # Derivative of the Weights as a Average
        db2 = np.sum(outputError, axis=0, keepdims=True) / trainDataSize

        dW1 = np.dot(trainData.T, hiddenError) / trainDataSize
        db1 = np.sum(hiddenError, axis=0, keepdims=True) / trainDataSize
        
        # L1 Regularization
        dW1 += self.lambdal1 * np.sign(self.W1)
        dW2 += self.lambdal1 * np.sign(self.W2)
                
        # L2 Regularization
        dW1 += self.lambdal2 * self.W1
        dW2 += self.lambdal2 * self.W2
        
        # Implying the Error contributions to the weights and biases
        
        self.W1 -= self.learnRate * dW1
        self.b1 -= self.learnRate * db1
        
        self.W2 -= self.learnRate * dW2
        self.b2 -= self.learnRate * db2
        
    def TrainModel(self, trainData, labelData, epochs):
        for epoch in range(epochs):
            prediction = self.ForwardPropogation(trainData)     # Performing the Forward Propogation for the Model
            self.BackwardPropogation(trainData, labelData, prediction)      # Performing the Backward Propogation for the Weights and Biases
            
            if(epoch % 1000 == 0):
                loss = -np.mean(np.sum(labelData * np.log(prediction + 1e-10), axis=1))
                print(f"Epoch: {epoch}, Loss: {loss: .4f}")     # Printing the Loss Value for every 100 Epochs
                
    def Predict(self, trainData):
        prediction = self.ForwardPropogation(trainData)
        return np.argmax(prediction, axis=1)

In [ ]:
# Defining the Hyperparameters

inputSize = 28*28   # The Number of Input Neurons -- Here it is 28*28, Because of the number of pixels
hiddenSize = 512    # The Number of the Hidden Neurons
outputSize = len(classNames)     # The Number of possible outputs -- Here it is the length of the number of classes of clothing
learnRate = 0.1     # The amount by which the error may decrease

# Regularization
l1 = 0.0001
l2 = 0.001

epochs = 25000      # The Number of generations that the model will go through

# Now actually defining a object of the Neural Network

This will actually train the Neural Network Model

In [ ]:
# Defining the Neural Network Model Object
neuralNetworkModel = NeuralNetwork(inputSize, hiddenSize, outputSize, learnRate, l1, l2)
neuralNetworkModel.TrainModel(trainImages, trainLabelsEncoded, epochs)

# Checking Accuracy of the Predictions
finalPredicition = neuralNetworkModel.Predict(testImages)
accuracy = np.mean(finalPredicition == testLabels)

print(f"Test Accuracy: {accuracy:.4f}")